### Pydantic Basics: Creating and Using Models
Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime

In [1]:
from dataclasses import dataclass

@dataclass
class Person():
    name: str
    age: int
    city: str

In [3]:
person = Person(name="Sooriya", age=20, city="Ohio")

print (person)

Person(name='Sooriya', age=20, city='Ohio')


In [4]:
person = Person(name="Sooriya", age=20, city=10)

print (person)

Person(name='Sooriya', age=20, city=10)


Using dataclass when the object creation the data type is not getting validated. Now let's try with Pydantic.

In [5]:
## Pydantic
from pydantic import BaseModel

In [6]:
class Person1(BaseModel):
    name: str
    age: int
    city: str

In [8]:
p1 = Person1(name="Kumar", age=15, city="Scotland")

print (p1)

name='Kumar' age=15 city='Scotland'


In [10]:
p2 = Person1(name="Kumar", age=15, city=100)

print (p2)

ValidationError: 1 validation error for Person1
city
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [11]:
p3 = Person1(name="Kumar", age=15, city="100")

print (p3)

name='Kumar' age=15 city='100'


Optional Fields

In [12]:
from typing import Optional

class Employee(BaseModel):
    id: int
    name: str
    department: str
    salary: Optional[float] = None
    is_active: Optional[bool] = True

In [14]:
emp1 = Employee(id=1, name="Alice", department="Development")

print (emp1)

id=1 name='Alice' department='Development' salary=None is_active=True


In [17]:
emp2 = Employee(id=1, name="Alice", department="Development", salary="do not know")

print (emp2)

ValidationError: 1 validation error for Employee
salary
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='do not know', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing

In [18]:
emp3 = Employee(id=1, name="Alice", department="Development", salary=10000)

print (emp3)

id=1 name='Alice' department='Development' salary=10000.0 is_active=True


In [19]:
emp4 = Employee(id=1, name="Alice", department="Development", salary="10000")

print (emp4)

id=1 name='Alice' department='Development' salary=10000.0 is_active=True


In [20]:
from typing import List

class Classroom(BaseModel):
    room_number: str
    students: List[str]
    capcity: int

In [23]:
classroom1 = Classroom(
    room_number = "C0001",
    students = ("Alice", "Bob", "Charlie"),
    capcity = 100
)

print (classroom1)

room_number='C0001' students=['Alice', 'Bob', 'Charlie'] capcity=100


In [24]:
classroom2 = Classroom(
    room_number = "C0002",
    students = ("Alice", 2, "Charlie"),
    capcity = 100
)

print (classroom2)

ValidationError: 1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [25]:
classroom3 = Classroom(
    room_number = "C0003",
    students = ("Alice", "2", "Charlie"),
    capcity = 100
)

print (classroom3)

room_number='C0003' students=['Alice', '2', 'Charlie'] capcity=100


Nested Models

In [26]:
from pydantic import BaseModel

class Address(BaseModel):
    street:str
    city:str
    zip_code:str

class Customer(BaseModel):
    customer_id:int
    name:str
    address:Address  ## Nested Model

In [31]:
c1 = Customer(customer_id=1, name="Alice", address={
    "street": "Main", "city": "Oakland", "zip_code": "12345"
})

print (c1)

customer_id=1 name='Alice' address=Address(street='Main', city='Oakland', zip_code='12345')


In [32]:
c2 = Customer(customer_id=1, name="Alice", address={
    "street": "Main", "city": "Oakland", "zip_code": 12345
})

print (c2)

ValidationError: 1 validation error for Customer
address.zip_code
  Input should be a valid string [type=string_type, input_value=12345, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

Pydantic Field

In [35]:
from pydantic import BaseModel, Field

class Item(BaseModel):
    name: str = Field(min_length=2, max_length=10)
    price: float = Field(gt=0, le=1000)
    quantity: int = Field(gt=0)

In [36]:
item1 = Item(name="Book", price=500, quantity=1)

print (item1)

name='Book' price=500.0 quantity=1


In [44]:
item2 = Item(name="Book", price=1001, quantity=10)

print (item2)

ValidationError: 1 validation error for Item
price
  Input should be less than or equal to 1000 [type=less_than_equal, input_value=1001, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

In [50]:
class User(BaseModel):
    user_name:str=Field(description="Unique user name for the user")
    age:int=Field(default=18,description="User age default to 18 ")
    email:str= Field(default_factory=lambda: "user@example.com",description="Default email address")

In [52]:
user1 = User(user_name="alice")
print(user1)

user_name='alice' age=18 email='user@example.com'


In [53]:
user2 = User(user_name="alice", age=20, email="alice@gmail.com")
print(user2)

user_name='alice' age=20 email='alice@gmail.com'


In [54]:
User.model_json_schema()

{'properties': {'user_name': {'description': 'Unique user name for the user',
   'title': 'User Name',
   'type': 'string'},
  'age': {'default': 18,
   'description': 'User age default to 18 ',
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'Default email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['user_name'],
 'title': 'User',
 'type': 'object'}